In [2]:
####################################################################################################
####################################################################################################
##########                      C	limate-Change                                       ############
##########                      A	ssessment Model for                                 ############
##########                      R	enewable Energy                                     ############
##########                      P	ower output                                         ############
##########                                                                              ############
####################################################################################################
####################################################################################################
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
import os
import pulp
import matplotlib.pyplot as plt
from scipy.stats import weibull_min
import tqdm
import os
from pathlib import Path

In [2]:
# Function to create directory structure
def setup_directory_structure(base_path):
    scenarios = ['RCP2.6', 'RCP4.5', 'RCP8.5']
    subfolders = ['pv_interim', 'wind_interim', 'Results']

    # Iterate over each subfolder and scenario to create the directory tree
    for subfolder in subfolders:
        for scenario in scenarios:
            # Construct the full path for the directory
            dir_path = Path(base_path) / 'Data' / subfolder / scenario
            # Create the directory if it doesn't exist
            os.makedirs(dir_path, exist_ok=True)


base_path = Path(os.getcwd())


# Setup the directory structure based on the base location
setup_directory_structure(base_path)


In [7]:
power_curves = pd.read_csv(f'powercurves.csv', sep=	';')
germany_gdf_template = None
plz_shapefile=gpd.read_file('georef-germany-postleitzahl.shp')

# Create a DataFrame from 2023 to 2050 with each year increasing by 10000 in expansion
pathways = {
    "Year": [2021,2022,2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050],
    "Wind On-Shore Expansion": [0.0,0.0,0.0, 9.2, 7.5, 7.5, 7.5, 7.5, 8.0, 8.0, 8.4, 8.4, 8.4, 8.4, 8.4, 0.6, 0.6, 0.6, 0.6, 0.6,0,0,0,0,0,0,0,0,0,0]
}
expansion_df = pd.DataFrame({
    "Year": pathways["Year"],
    "Wind On-Shore Expansion": pathways["Wind On-Shore Expansion"]
})
expansion_df['Wind On-Shore Expansion'] = expansion_df['Wind On-Shore Expansion'] * 1000
expansion_df.to_csv(f'Data/expansion2023.csv', sep=';', index=False)





In [ ]:

def process_shapefiles_with_wind_data_optimized(plz_shapefile, wind_gdf):
        
        # Read the PLZ shapefile
        plz_gdf = plz_shapefile.copy()
        #adjust the crs to a metric crs as this will help the buffer
        wind_gdf.crs = 'epsg:4647'
        plz_gdf.crs='epsg:4647'
        

        # Define time columns and initialize new columns in plz_gdf
        time_cols = [col for col in wind_gdf.columns if col not in ['geometry']]
        for time_col in time_cols:
            plz_gdf[f'wind_speed_mean_{time_col}'] = None
        plz_gdf['point_ids'] = None
        plz_gdf['point_count'] = 0
        plz_gdf['type'] = None
        plz_gdf['closest_point_distance'] = None

        # Spatial join for points within polygons
        joined_gdf = gpd.sjoin(plz_gdf, wind_gdf, how='left', op='contains')
        for idx, group in joined_gdf.groupby(joined_gdf.index):
            wind_points = group.dropna(subset=['ID'])
            plz_gdf.at[idx, 'point_ids'] = list(wind_points['ID'].astype(int))
            plz_gdf.at[idx, 'point_count'] = len(wind_points)
            plz_gdf.at[idx, 'type'] = 'within' if len(wind_points) > 0 else None
            for time_col in time_cols:
                plz_gdf.at[idx, f'wind_speed_mean_{time_col}'] = wind_points[time_col].mean()

        # Optimized handling of PLZ polygons without wind points
        no_wind_plz = plz_gdf[plz_gdf['point_count'] == 0]
        buffer_distance = 14000  # 14 km buffer
        wind_sindex = wind_gdf.sindex  # Spatial index for wind_gdf

        for idx, row in no_wind_plz.iterrows():
            # Use spatial index to narrow down the candidates
            potential_matches_index = list(wind_sindex.intersection(row.geometry.buffer(buffer_distance).bounds))
            potential_matches = wind_gdf.iloc[potential_matches_index]
            # Calculate the nearest point
            nearest_point_data = potential_matches.distance(row.geometry).idxmin()
            nearest_point = wind_gdf.loc[nearest_point_data] if pd.notna(nearest_point_data) else None

            if nearest_point is not None:
                nearest_point_id = nearest_point.name
                plz_gdf.at[idx, 'point_ids'] = [nearest_point_id]
                plz_gdf.at[idx, 'point_count'] = 1
                plz_gdf.at[idx, 'type'] = 'nearby'
                plz_gdf.at[idx, 'closest_point_distance'] = row.geometry.distance(nearest_point.geometry)
                for time_col in time_cols:
                    plz_gdf.at[idx, f'wind_speed_mean_{time_col}'] = nearest_point[time_col]
        return plz_gdf



In [ ]:
def create_germany_gdf_from_ds(ds, lat_min=46, lat_max=56, lon_min=5, lon_max=16):
    """
    Creates a GeoDataFrame containing points within Germany's geographic bounds from a given dataset.
    
    Parameters:
    - ds: The dataset containing latitude, longitude, and wind speed data.
    - lat_min, lat_max, lon_min, lon_max: Geographic bounds for filtering the points.
    
    Returns:
    - A GeoDataFrame with points within the specified bounds and wind speed data for each time step.
    """
    # Create a DataFrame from dataset coordinates
    lat_lon_df = pd.DataFrame({
        'lat': ds['lat'].values.ravel(),
        'lon': ds['lon'].values.ravel()
    })

    # Create a GeoDataFrame from lat_lon_df
    gdf = gpd.GeoDataFrame(lat_lon_df, geometry=gpd.points_from_xy(lat_lon_df.lon, lat_lon_df.lat))

    # Add each time step as a new column with simplified name 'year-month'
    for time in ds['time'].values:
        simplified_time_name = pd.to_datetime(time).strftime('%Y-%m')
        data_slice = ds.sel(time=time)
        gdf[simplified_time_name] = data_slice['sfcWind'].values.ravel()

    # Filter the GeoDataFrame for points within the specified geographic bounds
    germany_gdf = gdf[(gdf['lat'] >= lat_min) & (gdf['lat'] <= lat_max) & (gdf['lon'] >= lon_min) & (gdf['lon'] <= lon_max)]
    germany_gdf['ID'] = germany_gdf.index
    
    return germany_gdf

In [ ]:

def process_shapefiles_with_wind_data_optimized(plz_shapefile, wind_gdf):
        
        # Read the PLZ shapefile
        plz_gdf = plz_shapefile.copy()
        #adjust the crs to a metric crs as this will help the buffer
        wind_gdf.crs = 'epsg:4647'
        plz_gdf.crs='epsg:4647'
        

        # Define time columns and initialize new columns in plz_gdf
        time_cols = [col for col in wind_gdf.columns if col not in ['geometry']]
        for time_col in time_cols:
            plz_gdf[f'wind_speed_mean_{time_col}'] = None
        plz_gdf['point_ids'] = None
        plz_gdf['point_count'] = 0
        plz_gdf['type'] = None
        plz_gdf['closest_point_distance'] = None

        # Spatial join for points within polygons
        joined_gdf = gpd.sjoin(plz_gdf, wind_gdf, how='left', op='contains')
        for idx, group in joined_gdf.groupby(joined_gdf.index):
            wind_points = group.dropna(subset=['ID'])
            plz_gdf.at[idx, 'point_ids'] = list(wind_points['ID'].astype(int))
            plz_gdf.at[idx, 'point_count'] = len(wind_points)
            plz_gdf.at[idx, 'type'] = 'within' if len(wind_points) > 0 else None
            for time_col in time_cols:
                plz_gdf.at[idx, f'wind_speed_mean_{time_col}'] = wind_points[time_col].mean()

        # Optimized handling of PLZ polygons without wind points
        no_wind_plz = plz_gdf[plz_gdf['point_count'] == 0]
        buffer_distance = 14000  # 14 km buffer
        wind_sindex = wind_gdf.sindex  # Spatial index for wind_gdf

        for idx, row in no_wind_plz.iterrows():
            # Use spatial index to narrow down the candidates
            potential_matches_index = list(wind_sindex.intersection(row.geometry.buffer(buffer_distance).bounds))
            potential_matches = wind_gdf.iloc[potential_matches_index]
            # Calculate the nearest point
            nearest_point_data = potential_matches.distance(row.geometry).idxmin()
            nearest_point = wind_gdf.loc[nearest_point_data] if pd.notna(nearest_point_data) else None

            if nearest_point is not None:
                nearest_point_id = nearest_point.name
                plz_gdf.at[idx, 'point_ids'] = [nearest_point_id]
                plz_gdf.at[idx, 'point_count'] = 1
                plz_gdf.at[idx, 'type'] = 'nearby'
                plz_gdf.at[idx, 'closest_point_distance'] = row.geometry.distance(nearest_point.geometry)
                for time_col in time_cols:
                    plz_gdf.at[idx, f'wind_speed_mean_{time_col}'] = nearest_point[time_col]
        return plz_gdf



In [ ]:
#germany_gdf=create_germany_gdf_from_ds(ds)


In [ ]:
def process_year_for_germany(germany_gdf_template, paths):
    results = []
    
    for path in paths:
        # Load the dataset
        ds = xr.open_dataset(path)
        
        # Create a base DataFrame from the coordinates (if not already created)
        if germany_gdf_template is None:
            lat_lon_df = pd.DataFrame({
                'lat': ds['lat'].values.ravel(),
                'lon': ds['lon'].values.ravel()
            })
            germany_gdf = gpd.GeoDataFrame(lat_lon_df, geometry=gpd.points_from_xy(lat_lon_df.lon, lat_lon_df.lat))
        else:
            germany_gdf = germany_gdf_template.copy()
        
        # Extract all years data for 'sfcWind'
        for time in ds['time'].values:
            simplified_time_name = pd.to_datetime(time).strftime('%Y-%m')
            data_slice = ds.sel(time=time)
            germany_gdf[simplified_time_name] = data_slice['sfcWind'].values.ravel()
        
        # Add dataset identifier based on the path (to distinguish between RCP scenarios)
        rcp_id = path.split('_')[3]  # Assumes RCP info is the 4th element in the filename
        germany_gdf['RCP'] = rcp_id
        
        results.append(germany_gdf)
    
    return results


In [ ]:
#TODO: include RCP stuff!!
# Assuming you have defined the function as provided in the previous code snippet

# Define the paths to your NetCDF files for each RCP scenario
paths = [
    'get_data/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
    'get_data/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp45_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
    'get_data/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc'
]

# Specify the year of interest
#year_oi = 2024  # Example year

# Initial call to the function (assuming germany_gdf_template is None for the first call)
germany_gdf_template = None  # This will be generated inside the function

# Call the function with the specified year and paths
#processed_data = process_year_for_germany(year_oi, germany_gdf_template, paths)


In [ ]:
def process_year(germany_gdf,scenario):
    # Load PLZ shapefile
    plz = gpd.read_file('georef-germany-postleitzahl.shp')

    # Assuming the process_shapefiles_with_wind_data_optimized returns the full DataFrame with multiple years data
    updated_plz_gdf = process_shapefiles_with_wind_data_optimized(plz, germany_gdf)
    print("updated_plz_gdf erzeugt")

    #Drop unnecessary columns
    #updated_plz_gdf.drop(['name', 'plz_name', 'plz_name_lo', 'krs_code', 'lan_name', 'lan_code', 'krs_name', 'PLZ_int'], axis=1, inplace=True)
    
    # Rename columns as needed
    updated_plz_gdf.rename(columns={'wind_speed_mean_ID':'ID', 'wind_speed_mean_lat':'lat', 'wind_speed_mean_lon':'lon'}, inplace=True)
    # Select only the necessary columns
    #updated_plz_gdf = updated_plz_gdf[['plz_code', 'BL', 'GrPow_MW', 'Anzahl_Anlagen', 'avg_RD', 'avg_NH']]

    # Remove duplicates
    updated_plz_gdf.drop_duplicates(subset='plz_code', keep='first', inplace=True)

    # Extract years from column names and save separate CSVs for each year
    years = set(col.split('-')[0] for col in updated_plz_gdf.columns if '-' in col)
    for year in years:
        # Select columns for the specific year along with ID, plz_code, lat, lon, and geometry
        columns = ['ID', 'plz_code', 'lat', 'lon', 'geometry'] + [col for col in updated_plz_gdf.columns if col.startswith(year)]
        year_df = updated_plz_gdf[columns]

        # Save the DataFrame for this year into a CSV file
        csv_path = f'Data/wind_interimupdated_plz_{scenario}_{year}.csv'
        year_df.to_csv(csv_path, index=False)
        print(f"Data for {year} saved to {csv_path}")

    return updated_plz_gdf


#updated_plz_gdf = process_year(year_oi, germany_gdf)


In [15]:
def calculate_annual_contraction(start_year, end_year, input_dir='Data/', output_dir='Data/wind_interimcontraction'):
    # Load the combined MASTR data
    file_path = f'{input_dir}/2023_MASTR_ALR.csv'
    mastr_data = pd.read_csv(file_path, sep=';')
    mastr_data['INBETRIEBNAHMEDATUM'] = pd.to_datetime(mastr_data['INBETRIEBNAHMEDATUM'])
    mastr_data = mastr_data.dropna(subset=['INBETRIEBNAHMEDATUM'])
    mastr_data = mastr_data[mastr_data.EINHEITBETRIEBSSTATUS != 'EndgueltigStillgelegt']
    # Loop through each year to calculate decommissioning for exactly 20-year-old turbines
    for year in range(start_year, end_year + 1):
        # Filter turbines that turn exactly 20 years old during the year
        yearly_decommissioned = mastr_data[mastr_data['IBN_YEAR']== (year - 20)]
        

        # Aggregate the decommissioned data by postcode
        aggregated_data = yearly_decommissioned.groupby('POSTLEITZAHL').agg(
            Total_BRUTTOLEISTUNG=('BRUTTOLEISTUNG', 'sum'),
            Anzahl_Anlagen=('POSTLEITZAHL', 'count'),
            avg_RD=('RD_NEW', 'mean'),  # Assuming 'RD_NEW' is the column for Rotor Diameter
            avg_NH=('NH_new', 'mean'),  # Assuming 'NH_new' is the column for Hub Height
            Bundesland=('BUNDESLAND', 'first')
        ).reset_index()

        #rename POSTLEITZAHL to plz_code
        aggregated_data.rename(columns={'POSTLEITZAHL':'plz_code'}, inplace=True)
        aggregated_data.rename(columns={'Total_BRUTTOLEISTUNG':'GrPow_MW'}, inplace=True)
        # Save the yearly data
        aggregated_data.to_csv(f'{output_dir}/contraction_data_{year}.csv', sep=';', index=False)
        print(f'Contraction data for year {year} saved')





In [29]:
def prepare_height_df( year_oi, input_dir='Data/', output_dir='Data/Wind_interim'):
    """
    Loads MaStR data for the specified year, prepares contraction data, and saves the processed DataFrame.
    Parameters:
    - year_befoistr: The year before the year of interest, as a string or integer.
    - year_oi: The year of interest, used for calculating the age of installations, as a string or integer.
    - input_dir: Directory path where the input data files are located.
    - output_dir: Directory path where the output data files will be saved.
    Returns:
    - DataFrame that includes turbine and power data aggregated by postcode.
    """
    # Load expansion data for the specified year
 
    # Load MaStR data
    file_path = f'{input_dir}/2023_MASTR_ALR.csv'
    contraction = pd.read_csv(file_path, sep=';')

    # Select relevant columns and clean data
    contraction = contraction[['EINHEITMASTRNUMMER','BUNDESLAND', 'LANDKREIS', 'GEMEINDE', 'POSTLEITZAHL', 'MELDEDATUM', 'INBETRIEBNAHMEDATUM', 'GEPLANTESINBETRIEBNAHMEDATUM', 'EINHEITBETRIEBSSTATUS', 'BRUTTOLEISTUNG', 'NH_new', 'RD_NEW', 'UTM32_EAST', 'UTM32_NORTH']]
    contraction['INBETRIEBNAHMEDATUM'] = pd.to_datetime(contraction['INBETRIEBNAHMEDATUM']).fillna(pd.to_datetime(contraction['GEPLANTESINBETRIEBNAHMEDATUM'], errors='coerce',dayfirst=True))
    #contraction = contraction.dropna(subset=['INBETRIEBNAHMEDATUM'])
    contraction = contraction[contraction.EINHEITBETRIEBSSTATUS != 'EndgueltigStillgelegt']
    contraction['Age_as_of_year_oi'] = year_oi - contraction['INBETRIEBNAHMEDATUM'].dt.year
    contraction = contraction[contraction.Age_as_of_year_oi <= 21]
   


    # Aggregate contraction data by PLZ for relevant metrics
    height_df = contraction.groupby('POSTLEITZAHL').agg(
        BL=('BUNDESLAND', 'first'),
        GrPow_MW=('BRUTTOLEISTUNG', 'sum'),
        Anzahl_Anlagen=('POSTLEITZAHL', 'count'),
        avg_RD=('RD_NEW', 'mean'),
        avg_NH=('NH_new', 'mean')
    ).reset_index()

    # Update column names and convert power from kW to MW
    height_df.rename(columns={'POSTLEITZAHL': 'plz_code', 'GrPow_MW': 'GrPow_MW'}, inplace=True)
    height_df.to_csv(f'{output_dir}/height_data_2023.csv', sep=';', index=False )

    return height_df




In [42]:
def distribute_expansion_randomly(year,expansion_chunk_value):
    year_befoistr=str(year)
    # Define the decrease in area per chunk distributed
    year_decom=year+20

    blpath='Data/BL-Area_'+year_befoistr+'.csv'
    BL_Area=pd.read_csv(blpath,sep=';')
    height_df=pd.read_csv(f'Data/wind_interim/height_data_{year}.csv',sep=';')

    height_df=height_df.replace([np.inf, -np.inf], np.nan)
    height_df=height_df.dropna(subset=['avg_RD'])
    height_df=height_df.dropna(subset=['avg_NH'])
    height_df=height_df.dropna(subset=['GrPow_MW'])

    average_avg_RD = height_df['avg_RD'].mean()
    average_avg_NH = height_df['avg_NH'].mean()
    rd_value=3.48498987*year-6915.90402785 #Regression line for RD from Mastr data, RMSE: 15.8333, R2: 0.7646
    nh_value=0.8523*rd_value+26.6221 #Regression line for NH from Mastr data, RMSE: 19.71183, R2: 0.66930
    expansion_df=pd.read_csv(f'Data/expansion2023.csv',sep=';')
    contraction_df=pd.read_csv(f'Data/wind_interim/contraction_data_{year}.csv',sep=';')
    
    area_decrease_per_chunk = 3.14159265*((average_avg_RD**2)+900)/1000000


    add_to_expansion=contraction_df['GrPow_MW'].sum()
    
    area_increase = contraction_df.groupby('Bundesland').agg(
        Mean_RD_Contraction=('avg_RD', 'mean'),
        Total_Anlagen_Contraction=('Anzahl_Anlagen', 'sum')
    ).reset_index()

    # Calculate the increase in area for each Bundesland based on the mean RD contraction
    area_increase['AreaIncrease'] = 3.14159265 * ((area_increase['Mean_RD_Contraction']**2) + 900) / 1000000

    # Multiply the 'AreaIncrease' by the number of contracted installations to get total area to increase
    area_increase['TotalAreaIncrease'] = area_increase['AreaIncrease'] * area_increase['Total_Anlagen_Contraction']    
    
    # Setting index for BL_Area if not already done
    if BL_Area.index.name != 'Bundesland':
        BL_Area.set_index('Bundesland', inplace=True)

    #add the area increase to the BL_Area DataFrame
    for index, row in area_increase.iterrows():
        BL_Area.at[row['Bundesland'], 'Freie Flaeche'] += row['TotalAreaIncrease']




    total_expansion_value1=expansion_df[expansion_df['Year']==year]['Wind On-Shore Expansion'].values[0]
    total_expansion_value=add_to_expansion+total_expansion_value1
    #print(f'total_expansion_value: {total_expansion_value}, add_to_expansion: {add_to_expansion}, EEG_expansion_value: {total_expansion_value1}  ')

    # Merge the dataframes on 'plz_code'
    merged_df = pd.merge(height_df, contraction_df, on='plz_code', how='left', suffixes=('', '_contraction'))

    # Fill NaN values with 0 for contraction data (assuming no contraction means 0)
    merged_df['GrPow_MW_contraction'].fillna(0, inplace=True)
    merged_df['Anzahl_Anlagen_contraction'].fillna(0, inplace=True)
    merged_df['avg_RD_contraction'].fillna(0, inplace=True)
    merged_df['avg_NH_contraction'].fillna(0, inplace=True)  # Fill with average if no data
    
    # Calculate total contributions before contraction
    merged_df['total_RD_before'] = merged_df['avg_RD'] * merged_df['Anzahl_Anlagen']
    merged_df['total_NH_before'] = merged_df['avg_NH'] * merged_df['Anzahl_Anlagen']
    # Subtract the contraction values
    merged_df['GrPow_MW'] = merged_df['GrPow_MW'] - merged_df['GrPow_MW_contraction']
    # print(f'GrPow_MW after subtraction: {merged_df["GrPow_MW"].sum()}')
    merged_df['Anzahl_Anlagen'] = merged_df['Anzahl_Anlagen'] - merged_df['Anzahl_Anlagen_contraction']
    merged_df['total_RD_after'] = merged_df['total_RD_before'] - (merged_df['avg_RD_contraction'] * merged_df['Anzahl_Anlagen_contraction'])
    merged_df['total_NH_after'] = merged_df['total_NH_before'] - (merged_df['avg_NH_contraction'] * merged_df['Anzahl_Anlagen_contraction'])
    
 
    # Calculate avg_RD and avg_NH, defaulting to 0 when Anzahl_Anlagen is 0
    merged_df['avg_RD'] = np.where(merged_df['Anzahl_Anlagen'] > 0, 
                                merged_df['total_RD_after'] / merged_df['Anzahl_Anlagen'], 
                                0)
    merged_df['avg_NH'] = np.where(merged_df['Anzahl_Anlagen'] > 0, 
                                merged_df['total_NH_after'] / merged_df['Anzahl_Anlagen'], 
                                0)
    #print smallest value of avg_rd, avg_NH and GrPow_MW
    # print(f'Smallest avg_RD: {merged_df["avg_RD"].min()}')
    # print(f'Smallest avg_NH: {merged_df["avg_NH"].min()}')
    # print(f'Smallest GrPow_MW: {merged_df["GrPow_MW"].min()}')
    
    # Drop the contraction columns if they are no longer needed
    merged_df.drop(columns=['GrPow_MW_contraction', 'Anzahl_Anlagen_contraction', 'avg_RD_contraction', 'avg_NH_contraction', 'total_RD_before', 'total_NH_before', 'total_RD_after', 'total_NH_after'], inplace=True)

    height_df=merged_df #Elias empfiehlt: merged_df.copy()


    print(f'GrPow_MW as HDF: {height_df["GrPow_MW"].sum()}')
    
    # Create a new LP problem
    prob = pulp.LpProblem("RandomDistributionOfExpansion", pulp.LpMaximize)

    # Total number of chunks available
    total_chunks = int(total_expansion_value / expansion_chunk_value)
    # print(f'total_chunks: {total_chunks}')
    # print(f"Year of Interest: {year}, Selected Expansion Value: {total_expansion_value}")
    # Variables: Number of chunks to distribute to each plz_code
    chunks = pulp.LpVariable.dicts("Chunks",
                                   (index for index in height_df.index),
                                   lowBound=0,
                                   upBound=4,  #max 10 installations per plz_code
                                   cat='Integer')

    # Constraint: Sum of all distributed chunks should not exceed total available chunks
    prob += pulp.lpSum([chunks[i] for i in height_df.index]) <= total_chunks, "TotalExpansionLimit"
   
    # Area constraints for each plz_code
    for index, row in height_df.iterrows():
        bl = row['BL']  # Get the region identifier
        # Maximum area available for this plz_code, fetched from BL_Area DataFrame
        available_area = BL_Area.at[bl, 'Freie Flaeche']
        # Ensuring that the distributed units do not exceed the available area
        prob += chunks[index] * area_decrease_per_chunk <= available_area, f"AreaConstraint_{index}"
    
    # Objective: Distribute as evenly as possible, maximize the number of plz_code receiving chunks
    prob += pulp.lpSum([chunks[i] for i in height_df.index])

    # Solve the problem
    prob.solve()

    
   # Output results and update logic
    distribution_records = []
    if pulp.LpStatus[prob.status] == 'Optimal':
        # print("Distribution Completed Successfully")
        # print("Solver Status:", pulp.LpStatus[prob.status])
        
        total_distributed_chunks = sum(int(var.value()) for var in chunks.values())
        print(f"Total distributed chunks: {total_distributed_chunks} out of available {total_chunks}")
        
        accumulated_distributed_chunks = 0  # Initialize at 0 to accumulate correctly
        accumulated_distributed_power = 0  # Initialize at 0 to accumulate the total power added

        for index, row in height_df.iterrows():
            distributed_chunks = int(pulp.value(chunks[index]))
            
            accumulated_distributed_chunks += distributed_chunks  # Accumulate distributed chunks
            
            if distributed_chunks > 0:
                additional_power = distributed_chunks * expansion_chunk_value
                height_df.at[index, 'Anzahl_Anlagen'] += distributed_chunks
                height_df.at[index, 'GrPow_MW'] += additional_power
                # Recalculate averages based on new totals
                total_rd = height_df.at[index, 'avg_RD'] * (height_df.at[index, 'Anzahl_Anlagen'] - distributed_chunks)
                total_nh = height_df.at[index, 'avg_NH'] * (height_df.at[index, 'Anzahl_Anlagen'] - distributed_chunks)
                total_rd += rd_value * distributed_chunks
                total_nh += nh_value * distributed_chunks
                height_df.at[index, 'avg_RD'] = total_rd / height_df.at[index, 'Anzahl_Anlagen']
                height_df.at[index, 'avg_NH'] = total_nh / height_df.at[index, 'Anzahl_Anlagen']
                accumulated_distributed_power += additional_power  # Add the power for these chunks to the total
                distribution_records.append({
                    'plz_code': row['plz_code'],
                    'Bundesland': row['BL'],
                    'GrPow_MW': additional_power,
                    'Anzahl_Anlagen': distributed_chunks,
                    'avg_RD': rd_value,
                    'avg_NH': nh_value
                })
        newsum = height_df['GrPow_MW'].sum()
        # print(f'maximum Gross Power: {height_df['GrPow_MW'].max()}')
        # print(f"Optimization was successful. New Gross Power (MW): {newsum} MW")
        # print(f"Chunks distributed: {accumulated_distributed_chunks} out of {total_chunks}")
        # print(f"Total power added: {accumulated_distributed_power} MW")
        height_df.to_csv(f'Data/wind_interimheight_data_{year+1}.csv', sep=';', index=False)
        distribution_records = pd.DataFrame(distribution_records)
        distribution_records.to_csv(f'Data/wind_interim/contraction_data_{year_decom}.csv', sep=';', index=False)
    else:
        print("Failed to find an optimal solution.")
    return distribution_records

In [ ]:

def process_wind_speed_data(scenario,year_oi, year_oistr, output_dir):
    """
    Processes wind speed data, interpolates wind speed at hub height, and saves the data to CSV files.

    Parameters:
    - updated_plz_gdf: GeoDataFrame with PLZ data updated.
    - height_df: DataFrame containing height data for wind turbines.
    - year_oistr: Year of interest as a string.
    - output_dir: Directory path where the output CSV files will be saved.
    """
    height_df=pd.read_csv(f'Data/wind_interimheight_data_{year_oi}.csv', sep=';')
    updated_plz_gdf=pd.read_csv(f'Data/wind_interimupdated_plz_{scenario}_wind_speed_mean_{year_oi}.csv')
    #drop rows in height df where Bruttoleistung, avg_NH or avg_RD is nan
    height_df=height_df.replace([np.inf, -np.inf], np.nan)
    height_df=height_df.dropna(subset=['avg_RD'])
    height_df=height_df.dropna(subset=['avg_NH'])
    height_df=height_df.dropna(subset=['GrPow_MW'])
    height_df.dropna(subset=['GrPow_MW', 'avg_NH', 'avg_RD','plz_code'], inplace=True)


    updated_plz_gdf['plz_code'] = updated_plz_gdf['plz_code'].astype(int)
    height_df['plz_code'] = height_df['plz_code'].astype(int)
    height_df['avg_NH'] = height_df['avg_NH'].astype(int)
    # Merge dataframes on 'plz_code'
    merged_df = pd.merge(updated_plz_gdf, height_df, on='plz_code', how='left')

    # Replace NaN with 0 in 'avg_NH'
    merged_df['avg_NH'].fillna(0, inplace=True)

    # Create the 'hellmann' column
    merged_df['hellmann'] = (merged_df['avg_NH'] / 10) ** 0.143

    # Process wind speed columns
    wind_speed_columns = merged_df.columns[merged_df.columns.str.startswith('wind_speed_mean_')]
    merged_df[wind_speed_columns] = merged_df[wind_speed_columns].apply(pd.to_numeric, errors='coerce')
    merged_df[wind_speed_columns] = merged_df[wind_speed_columns].multiply(merged_df['hellmann'], axis=0)

    # Prepare final DataFrame
    ws_height_df = merged_df[['plz_code', 'avg_NH', 'GrPow_MW', 'avg_RD', 'ID', 'Anzahl_Anlagen'] + list(wind_speed_columns)]

    # Fill NaN values with 0 and replace inf/-inf with 0
    ws_height_df.fillna(0, inplace=True)
    ws_height_df.replace([np.inf, -np.inf], 0, inplace=True)

    # Convert columns to int
    ws_height_df[['avg_NH', 'avg_RD', 'ID']] = ws_height_df[['avg_NH', 'avg_RD', 'ID']].astype(int)

    print("Wind speed at hub height interpolated")
    # delete double entries
    ws_height_df.drop_duplicates(subset='plz_code', keep='first', inplace=True)
    # Save DataFrames to CSV
    wind_speed_csv_path = f'{output_dir}/wind_speed_hub_height_{scenario}_{year_oi}.csv'
    ws_height_df.to_csv(wind_speed_csv_path, sep=';')


    
    return ws_height_df


In [ ]:
#ws_height_df=process_wind_speed_data(updated_plz_gdf, height_df, year_oistr, output_dir='Data')

In [44]:
def weibull_probability_vectorized(shape, scale, bounds):
    lower_bounds, upper_bounds = bounds[:-1], bounds[1:]
    return weibull_min.cdf(upper_bounds, shape, scale=scale) - weibull_min.cdf(lower_bounds, shape, scale=scale)



In [45]:

def calculate_production(power_curves, year_oi,scenario, plz_shapefile):
    from tqdm import tqdm
 
    """
    Calculates the wind energy production based on Weibull parameters and wind speed data.

    Parameters:
    - ws_height_df: DataFrame containing wind speed data at hub height.
    - weibull_params_df: DataFrame containing Weibull parameters by PLZ, year, and month.
    - power_curves: DataFrame containing power curves data.
    - year_oistr: Year of interest as a string.
    - output_dir: Directory path where the output CSV file will be saved.

    Returns:
    - Total wind energy production in TWh for the specified year.
    """
    year_oistr=str(year_oi)
    ws_height_df=pd.read_csv(f'Data/wind_interim/wind_speed_hub_height_{scenario}_{year_oi}.csv',sep=';')
    # Reshape ws_height_df to a long format for wind speeds
    weibull_params_df= pd.read_csv(f'Data/Weibull/{scenario}/weibull_params_{year_oi}.csv',sep=',')
    # Grouping by year, month, and windzone and calculating the mean of shape and scale
    weibull_params_df = weibull_params_df.groupby(['year', 'month', 'windzone']).agg({'shape': 'mean', 'scale': 'mean'}).reset_index()

    
    plz_shapefile=plz_shapefile[['plz_code', 'windzone']]
    
    ws_height_df['plz_code']=ws_height_df['plz_code'].astype(int)
    plz_shapefile['plz_code']=plz_shapefile['plz_code'].astype(int)
    ws_height_df=pd.merge(ws_height_df,plz_shapefile, on='plz_code', how='left')
    # Merge Weibull parameters

   
    
    wind_speed_cols = [col for col in ws_height_df.columns if col.startswith('wind_speed_mean')]
    
    # Reshape ws_height_df to a long format for wind speeds
    ws_height_long = ws_height_df.melt(id_vars=['ID', 'plz_code', 'windzone','avg_NH', 'GrPow_MW', 'avg_RD', 'Anzahl_Anlagen'], 
                                    value_vars=wind_speed_cols,
                                    var_name='month', value_name='WindSpeedMean')

    # Extract year and month
    ws_height_long['year'] = ws_height_long['month'].str[-7:-3]
    ws_height_long['month'] = ws_height_long['month'].str[-2:]
    ws_height_long['year'] = ws_height_long['year'].astype(int)
    ws_height_long['month'] = ws_height_long['month'].astype(int)

    # Merge ws_height_long and weibull_params_df
    merged_ws = pd.merge(ws_height_long, weibull_params_df, on=['month', 'year', 'windzone'], how='left')
    merged_ws = merged_ws[merged_ws['Anzahl_Anlagen'] != 0]

    # Prepare an empty DataFrame for storing probabilities
    probabilities_df = pd.DataFrame()

    # Define bounds for wind speeds
    bounds = power_curves['WS_m/s'].to_numpy()
    #scale_10m = mean_10m / np.exp(np.log(np.math.gamma(1 + 1/shape_10m)))
    # Calculate probabilities for each wind speed bin for each row in merged_ws
    for _, row in tqdm(merged_ws.iterrows(), total=merged_ws.shape[0], desc="Calculating Probabilities"):
        probabilities = weibull_probability_vectorized(row['shape']/2.5, row['scale'], bounds)
        
        # Prepare a DataFrame for the current row's probabilities
        temp_df = pd.DataFrame({
            'plz_code': row['plz_code'],
            'year': row['year'],
            'month': row['month'],
            'WS_m/s': bounds[:-1],
            'avg_RD': row['avg_RD'],
            'Anzahl_Anlagen': row['Anzahl_Anlagen'],
            'GrPow_MW': row['GrPow_MW'],
            'Probability': probabilities
        })
        
        # Append to the main probabilities DataFrame
        probabilities_df = pd.concat([probabilities_df, temp_df], ignore_index=True)

    # Save probabilities DataFrame to CSV
    probabilities_df.to_csv(f'Data/wind_interimwind_speed_probabilities_{scenario}_{year_oi}.csv', index=False, sep=';')
    # %%
    # Reshaping power_curves to long format
    power_curves_long = power_curves.melt(id_vars=['WS_m/s'], var_name='RD', value_name='Efficiency')

    # integer
    power_curves_long['RD'] = power_curves_long['RD'].astype(int)
    probabilities_df['avg_RD'] = probabilities_df['avg_RD'].astype(int)


    # Round avg_RD to the nearest multiple of 10 to match a power curve
    probabilities_df['avg_RD'] = (probabilities_df['avg_RD'] / 10).round() * 10



    # Merging probabilities_df with power_curves_long 
    probabilities_df = probabilities_df.merge(power_curves_long, left_on=['WS_m/s', 'avg_RD'], right_on=['WS_m/s', 'RD'], how='left')
    #delete rows with NaN values
    probabilities_df.dropna(subset=['Efficiency'], inplace=True)
    
    #drop the extra 'RD' column from the merge 
    probabilities_df.drop('RD', axis=1, inplace=True)




    # %%
    #scatter plot of probabilities_df
    import matplotlib.pyplot as plt
 

    # %%
    probabilities_df['Production']=probabilities_df['Probability']*probabilities_df['Efficiency']*probabilities_df['GrPow_MW']*720
    #print(f'mean of probabilities:  {probabilities_df["Probability"].mean()}')  
    #sum of production column
    tw=probabilities_df['Production'].sum()/1000000
    tw=str(tw)
    # Return total production
    total_production_twh = probabilities_df['Production'].sum() / 1000000
    print(f'Production in TWh: {total_production_twh} for year: {year_oi}')
    # Save probabilities DataFrame to CSV
    probabilities_df.to_csv(f'Data/wind_interimwind_speed_probabilities_{scenario}_{year_oi}.csv', index=False, sep=';')
    return probabilities_df


In [ ]:
prob_df32=pd.read_csv(f'Data/wind_interimwind_speed_probabilities_RCP26_2032.csv',  sep=';')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df=prob_df32
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt

# Assuming your DataFrame 'df' is already loaded.
# If it's in a CSV file, you can load it like this:
# df = pd.read_csv('path_to_your_file.csv')

# Grouping by 'WS_m/s' and summing up 'Probability'
grouped_data = df.groupby('WS_m/s')['Production'].sum()

# Plotting
plt.figure(figsize=(10, 5))
grouped_data.plot(kind='bar', color='purple')  # Using bar plot for clearer visualization
plt.title('Summed Production per WS_m/s')
plt.xlabel('WS_m/s (Wind Speed in meter per second 2032)')
plt.ylabel('Summed Production')
plt.grid(True)
plt.show()


In [ ]:
prob_df33['Testgroße']= prob_df33['GrPow_MW']*prob_df33['Probability'] 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df=prob_df33
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt

# Assuming your DataFrame 'df' is already loaded.
# If it's in a CSV file, you can load it like this:
# df = pd.read_csv('path_to_your_file.csv')

# Grouping by 'WS_m/s' and summing up 'Probability'
grouped_data = df.groupby('WS_m/s')['Production'].sum()

# Plotting
plt.figure(figsize=(10, 5))
grouped_data.plot(kind='bar', color='purple')  # Using bar plot for clearer visualization
plt.title('Summed Production per WS_m/s')
plt.xlabel('WS_m/s (Wind Speed in meter per second 2033)')
plt.ylabel('Summed Production')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your data
# Assuming your DataFrame is named 'df' and is already loaded. 
# If it's in a CSV file, you can load it like this:
# df = pd.read_csv('path_to_your_file.csv')
df=prob_df34
import pandas as pd
import matplotlib.pyplot as plt

# Assuming your DataFrame 'df' is already loaded.
# If it's in a CSV file, you can load it like this:
# df = pd.read_csv('path_to_your_file.csv')

# Grouping by 'WS_m/s' and summing up 'Probability'
grouped_data = df.groupby('WS_m/s')['Production'].sum()

# Plotting
plt.figure(figsize=(10, 5))
grouped_data.plot(kind='bar', color='purple')  # Using bar plot for clearer visualization
plt.title('Summed Production per WS_m/s')
plt.xlabel('WS_m/s (Wind Speed in meter per second) 2034')
plt.ylabel('Summed Production')
plt.grid(True)
plt.show()




In [ ]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the basemap
basemap_gdf = gpd.read_file('shp/georef-germany-postleitzahl/georef-germany-postleitzahl.shp')
start_year = 2021
end_year = 2050
scenarios = ['RCP26', 'RCP45', 'RCP85']

for scenario in scenarios:
    for year in range(start_year, end_year + 1):
        base_path = 'Data/Results/'
        file_namePV = f"PV_power_output_{scenario}_{year}.csv"
        file_nameWI = f"monthly_production_per_plz_{scenario}_{year}.csv"
        file_pathPV = os.path.join(base_path, file_namePV)
        file_pathWI = os.path.join(base_path, file_nameWI)

        if os.path.exists(file_pathPV) and os.path.exists(file_pathWI):
            dataPV = pd.read_csv(file_pathPV, sep=',')
            dataPV.drop_duplicates(subset=['plz_code', 'month'], inplace=True)
            dataPV['Production_PV'] = dataPV['Production'] / 1000  # Convert kW to MW
            
            dataWI = pd.read_csv(file_pathWI, sep=';')
            dataWI.drop_duplicates(subset=['plz_code', 'month'], inplace=True)
            dataWI.rename(columns={'Production': 'Production_wind'}, inplace=True)
            
            basemap_gdf['plz_code'] = basemap_gdf['plz_code'].astype(int)

            for month in range(1, 13):
                monthly_dataPV = dataPV[dataPV['month'] == month]
                monthly_dataWI = dataWI[dataWI['month'] == month]
                
                # Merge the data
                # Merge basemap_gdf with monthly_dataPV, filling missing production values with 1
                merged_gdf = basemap_gdf.merge(monthly_dataPV[['plz_code', 'Production_PV']], on='plz_code', how='left').fillna({'Production_PV': 1})

                merged_gdf = merged_gdf.merge(monthly_dataWI[['plz_code', 'Production_wind']], on='plz_code', how='left')
                merged_gdf['Total_Production'] = merged_gdf['Production_wind'].fillna(1) + merged_gdf['Production_PV'].fillna(1)

                # Fill NaNs and zeros with 1 for log scale plotting purposes
                merged_gdf['Production_wind'].replace(0, 1, inplace=True)
                merged_gdf['Production_wind'].fillna(1, inplace=True)

                # Plotting individual and combined production maps
                # PV Production Plot
                fig, ax = plt.subplots(1, figsize=(15, 10))
                merged_gdf.plot(column='Production_PV', ax=ax, legend=True,
                                norm=LogNorm(vmin=1, vmax=750000),
                                legend_kwds={'label': "PV Production by PLZ", 'orientation': "horizontal"},
                                cmap='viridis')
                plt.title(f'PV Production for {scenario} - {year}/{month}')
                plt.savefig(f'Data/Results/Maps/{scenario}/PV_{scenario}_{year}_month_{month}.png')
                plt.close()

                # Wind Production Plot
                fig, ax = plt.subplots(1, figsize=(15, 10))
                merged_gdf.plot(column='Production_wind', ax=ax, legend=True,
                                norm=LogNorm(vmin=1, vmax=750000),
                                legend_kwds={'label': "Wind Production by PLZ", 'orientation': "horizontal"},
                                cmap='viridis')
                plt.title(f'Wind Production for {scenario} - {year}/{month}')
                plt.savefig(f'Data/Results/Maps/{scenario}/Wind_{scenario}_{year}_month_{month}.png')
                plt.close()


                # Combined Production Plot
                fig, ax = plt.subplots(1, figsize=(15, 10))
                merged_gdf.plot(column='Total_Production', ax=ax, legend=True,
                                norm=LogNorm(vmin=1, vmax=750000),
                                legend_kwds={'label': "Total Production by PLZ", 'orientation': "horizontal"},
                                cmap='viridis')
                plt.savefig(f'Data/Results/Maps/{scenario}/Joined_{scenario}_{year}_{month}.png')
                plt.close()


In [ ]:

def monthly_production_save(year, scenario):
    probabilities_df=pd.read_csv(f'Data/wind_interimwind_speed_probabilities_{scenario}_{year}.csv', sep=';')


    # Assuming df is your DataFrame
    aggregated_df = probabilities_df.groupby(['plz_code', 'month']).agg({
        'WS_m/s': 'first',
        'avg_RD': 'first',
        'Anzahl_Anlagen': 'first',
        'GrPow_MW': 'first',
        'Probability': 'mean',
        'Efficiency': 'mean',
        'Production': 'sum'
    }).reset_index()
    aggregated_df

    plz_zerros=plz_shapefile['plz_code']
    plz_zerros=plz_zerros.astype(int)
    plz_zerros=pd.DataFrame(plz_zerros)
    plz_zerros
    #concatenate the plz_codes with the aggregated_df
    # Sample data setup (replace this with your actual DataFrame loading)
    # Assume 'aggregated_df' and 'plz_code_df' are already loaded as described

    # Check which PLZ codes are missing in the aggregated DataFrame
    missing_plz = plz_zerros[~plz_zerros['plz_code'].isin(aggregated_df['plz_code'])]

    # Create a DataFrame for missing PLZ codes with default values
    default_data = {
        'WS_m/s': 0,
        'avg_RD': 0,
        'Anzahl_Anlagen': 0,
        'GrPow_MW': 0,
        'Probability': 0,
        'Efficiency': 0,
        'Production': 0
    }
    months = list(range(1, 13))  # Months from 1 to 12

    # Using a list comprehension to build the DataFrame
    missing_rows = pd.DataFrame([
        {**{'plz_code': plz, 'month': month}, **default_data}
        for plz in missing_plz['plz_code'] for month in months
    ])

    # Concatenate the new rows with the original aggregated DataFrame
    final_df = pd.concat([aggregated_df, missing_rows], ignore_index=True)
    final_df.to_csv(f'Data/Results/monthly_production_per_plz_{scenario}_{year}.csv', index=False, sep=';')


In [ ]:
def add_and_save_production(scenario, year_oi, output_dir='Data/Results'):
    """
    adds the production of areas without installations (0 values) for depiction later saves the results along with monthly production to CSV files.

    Parameters:
    - updated_plz_gdf: GeoDataFrame with updated PLZ data.
    - probabilities_df: DataFrame with probabilities and production data.
    - ws_height_df: DataFrame with wind speed data at hub height.
    - year_oistr: Year of interest as a string.
    - output_dir: Directory path where the output CSV files will be saved.
    """
    updated_plz_gdf=pd.read_csv(f'Data/wind_interimupdated_plz_{scenario}_wind_speed_mean_{year_oi}.csv')
    probabilities_df=pd.read_csv(f'Data/wind_interimwind_speed_probabilities_{scenario}_{year_oi}.csv', sep=';')
    ws_height_df=pd.read_csv(f'Data/wind_interimwind_speed_hub_height_{scenario}_{year_oi}.csv', sep=';')
    print(ws_height_df.columns)  # This will show the exact column names as pandas sees them
    plz_codes = updated_plz_gdf['plz_code'].unique()
    plz_codes = pd.DataFrame(plz_codes, columns=['plz_code'])
    plz_codes = plz_codes.astype(int)
    
    months = range(1, 13)  # Months 1 through 12
    wind_speeds = np.arange(0, 25, 0.5)  # Wind speeds from 0 to 24.5 in 0.5 steps
    
    # Generate all combinations including wind speeds

    all_combinations_extended = pd.MultiIndex.from_product(
        [plz_codes['plz_code'], [year_oi], months, wind_speeds],  # Use plz_codes['plz_code'] here
        names=['plz_code', 'year', 'month', 'WS_m/s']
    ).to_frame(index=False)

    # Merge to include wind speed combinations and fill missing values
    probabilities_extended = pd.merge(
        all_combinations_extended, probabilities_df,
        on=['plz_code', 'year', 'month', 'WS_m/s'],
        how='left'
    )

    # Fill missing values
    columns_to_fill = ['avg_RD', 'Anzahl_Anlagen', 'GrPow_MW', 'Probability', 'Efficiency', 'Production']
    probabilities_extended[columns_to_fill] = probabilities_extended[columns_to_fill].fillna(0)

    # Aggregate production per PLZ and merge with additional data
    production_per_plz = probabilities_extended.groupby('plz_code')['Production'].sum().reset_index()
    production_per_plz = pd.merge(production_per_plz, ws_height_df[['plz_code', 'Anzahl_Anlagen', 'avg_RD', 'GrPow_MW', 'avg_NH']], on='plz_code', how='right')

    # Save aggregated production per PLZ
    production_per_plz.to_csv(f'{output_dir}/production_per_plz{scenario}_{year_oi}.csv', index=False)

    # Aggregate and save monthly production
    file_path = f'{output_dir}/monthly_production_per_plz_{scenario}_{year_oi}.csv'
    if os.path.exists(file_path):
        existing_data = pd.read_csv(file_path)
    else:
        existing_data = pd.DataFrame(columns=['plz_code', 'year', 'month', 'Production'])

    new_data = probabilities_df.groupby(['plz_code', 'year', 'month'])['Production'].sum().reset_index()
    updated_data = pd.concat([existing_data, new_data], ignore_index=True)
    updated_data.to_csv(file_path, index=False)

    print(f'Results saved for year: {year_oi}')
    return production_per_plz



In [ ]:
#production_per_plz= calculate_and_save_production(updated_plz_gdf, probabilities_df, ws_height_df, year_oistr, year_oi, output_dir='Data/Results')

In [ ]:
# Define RCP scenarios and years
import datetime
start_year = 2021
end_year = 2050
rcp_scenarios = {
    'RCP26': 'get_data/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
    'RCP45': 'get_data/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp45_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
    'RCP85': 'get_data/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
}


In [ ]:

# Loop through each RCP scenario
for rcp, rcp_path in rcp_scenarios.items():
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]:Processing for RCP scenario: {rcp}")
    ds=xr.open_dataset(rcp_path)
    germany_gdf=create_germany_gdf_from_ds(ds)
    print("germany_gdf erzeugt")
    processed_data = process_year_for_germany( germany_gdf_template, paths)
    updated_plz_gdf = process_year(germany_gdf, rcp)
    
    

In [ ]:
mastr='Data//2023_MASTR_ALR.csv'

In [ ]:
#for all years, 2023=Start year, 2050=end year  
prepare_height_df(2023, input_dir='Data/', output_dir='Data/Wind_interim')
calculate_annual_contraction(2023, 2050)


In [ ]:
start_year = 2023
end_year = 2050
for year in range(start_year, end_year + 1):
    print(f"Year: {year}")
    year_oi=year
    year_oistr=str(year_oi)
    year_befoistr=str(year_oi-1)
    expansion_chunk_value=0.2237*year-448.08
   #height_df=prepare_height_df(year_befoistr, year_oi, input_dir='Data/', output_dir='Data/Wind_interim')
    height_df=distribute_expansion_randomly(year, expansion_chunk_value)
    #print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Data preparation for year {year} completed.")
    


In [ ]:


# start_year = 2024
# end_year = 2049
scenarios = ['RCP26', 'RCP45', 'RCP85']

# for scenario in scenarios:
#     print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Processing for scenario {scenario}")
#     for year in range(start_year, end_year + 1):
#         ws_height_df = pd.read_csv(f'Data/wind_interimwind_speed_hub_height_{scenario}_{year}.csv', sep=';')
#         wind_speed_columns = [col for col in ws_height_df.columns if col.startswith('wind_speed_mean')]
#         # ws_height_df['weibull_params'] = ws_height_df.apply(lambda row: fitweibull(row[wind_speed_columns].values), axis=1)
#         # ws_height_df[['weibull_shape', 'weibull_scale']] = pd.DataFrame(ws_height_df['weibull_params'].tolist(), index=ws_height_df.index)
#         # # Extract only necessary columns for saving
#         # save_df = ws_height_df[['plz_code', 'weibull_shape', 'weibull_scale']]
#         # Save to CSV
#         output_filename = f"Data/Weibull/weibull_params_per_month_{scenario}_{year}.csv"
#         save_df.to_csv(output_filename, index=False)
#         print(f"Saved Weibull parameters to {output_filename}")
#         # Optionally, print the DataFrame to verify results
#         print(save_df.head())

In [ ]:
start_year = 2031
end_year = 2050
for rcp, rcp_path in rcp_scenarios.items():
    print(f"Final processing for RCP scenario: {rcp}")
    
    for year in range(start_year, end_year + 1):
        print(f"Final processing for year: {year}")
        year_oi=year
        year_oistr=str(year_oi)
        year_befoistr=str(year_oi-1)
        
        # Load or access ws_height_df and other necessary data produced in the first loop
        #weibull_params=pd.read_csv(f'Data/Weibull/weibull_params_per_month_{rcp}_{year_oi}.csv', sep=',')
        #weibull_params=prepare_Weibull(weibull_params, year_oi, rcp)
        
        #add a column with year and month. all year values are year, and month is from 1 to 12

        #probabilities_df = calculate_production(power_curves, year_oistr, output_dir='Data/Wind_interim')
        # generate_weibull_params(ws_height_df,plz_shapefile)
        #ws_height_df=process_wind_speed_data(rcp, year_oi, year_oistr, output_dir='Data/Wind_interim')
        # generate_weibull_params(ws_height_df,plz_shapefile)
        probabilities_df=calculate_production(power_curves, year_oi, rcp, plz_shapefile)
        production_per_plz = add_and_save_production(rcp,year_oi, output_dir='Data/Results') #deprecated
        monthly_production_save(year, rcp)

        print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Final analysis for year {year} and RCP scenario {rcp} completed.")
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Final analysis for RCP scenario {rcp} completed.")
print("All processing completed.")


In [ ]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the basemap
basemap_gdf = gpd.read_file('shp/georef-germany-postleitzahl/georef-germany-postleitzahl.shp')
start_year = 2023
end_year = 2050
scenarios = [ 'RCP85']

for scenario in scenarios:
    for year in range(start_year, end_year + 1):
        base_path = 'Data/Results/'
        file_namePV = f"PV_power_output_{scenario}_{year}.csv"
        file_nameWI = f"monthly_production_per_plz_{scenario}_{year}.csv"
        file_pathPV = os.path.join(base_path, file_namePV)
        file_pathWI = os.path.join(base_path, file_nameWI)

        if os.path.exists(file_pathPV) and os.path.exists(file_pathWI):
            dataPV = pd.read_csv(file_pathPV, sep=',')
            dataPV.drop_duplicates(subset=['plz_code', 'month'], inplace=True)
            dataPV['Production_PV'] = dataPV['Production'] / 1000  # Convert kW to MW
            
            dataWI = pd.read_csv(file_pathWI, sep=';')
            dataWI.drop_duplicates(subset=['plz_code', 'month'], inplace=True)
            dataWI.rename(columns={'Production': 'Production_wind'}, inplace=True)
            
            basemap_gdf['plz_code'] = basemap_gdf['plz_code'].astype(int)

            for month in range(1, 13):
                monthly_dataPV = dataPV[dataPV['month'] == month]
                monthly_dataWI = dataWI[dataWI['month'] == month]
                
                # Merge the data
                # Merge basemap_gdf with monthly_dataPV, filling missing production values with 1
                merged_gdf = basemap_gdf.merge(monthly_dataPV[['plz_code', 'Production_PV']], on='plz_code', how='left').fillna({'Production_PV': 1})

                merged_gdf = merged_gdf.merge(monthly_dataWI[['plz_code', 'Production_wind']], on='plz_code', how='left')
                merged_gdf['Total_Production'] = merged_gdf['Production_wind'].fillna(1) + merged_gdf['Production_PV'].fillna(1)

                # Fill NaNs and zeros with 1 for log scale plotting purposes
                merged_gdf['Production_wind'].replace(0, 1, inplace=True)
                merged_gdf['Production_wind'].fillna(1, inplace=True)

                # Plotting individual and combined production maps
                # PV Production Plot
                fig, ax = plt.subplots(1, figsize=(15, 10))
                merged_gdf.plot(column='Production_PV', ax=ax, legend=True,
                                norm=LogNorm(vmin=1, vmax=750000),
                                legend_kwds={'label': "PV Production by PLZ", 'orientation': "horizontal"},
                                cmap='viridis')
                plt.title(f'PV Production for {scenario} - {year}/{month}')
                plt.savefig(f'Data/Results/Maps/{scenario}/PV_{scenario}_{year}_month_{month}.png')
                plt.close()

                # Wind Production Plot
                fig, ax = plt.subplots(1, figsize=(15, 10))
                merged_gdf.plot(column='Production_wind', ax=ax, legend=True,
                                norm=LogNorm(vmin=1, vmax=750000),
                                legend_kwds={'label': "Wind Production by PLZ", 'orientation': "horizontal"},
                                cmap='viridis')
                plt.title(f'Wind Production for {scenario} - {year}/{month}')
                plt.savefig(f'Data/Results/Maps/{scenario}/Wind_{scenario}_{year}_month_{month}.png')
                plt.close()


                # Combined Production Plot
                fig, ax = plt.subplots(1, figsize=(15, 10))
                merged_gdf.plot(column='Total_Production', ax=ax, legend=True,
                                norm=LogNorm(vmin=1, vmax=750000),
                                legend_kwds={'label': "Total Production by PLZ", 'orientation': "horizontal"},
                                cmap='viridis')
                plt.savefig(f'Data/Results/Maps/{scenario}/Joined_{scenario}_{year}_{month}.png')
                plt.close()


In [3]:
import os
import pandas as pd

# Specify the directory containing the CSV files
directory = 'Data/wind_interim'

# Initialize a list to store the data
data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('wind_speed_hub_height') and filename.endswith('.csv'):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Extract the scenario and year from the filename   
        parts = filename.split('_')
        scenario = parts[4]  # Assumes 'RCP45' is always in the same position
        year = parts[5].split('.')[0]  # Split on '.' to remove the file extension
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep=';')
        
        # Check if 'GrPow_MW' column exists in the dataframe
        if 'GrPow_MW' in df.columns:
            # Filter out rows where 'GrPow_MW' is not 0
            #df_filtered = df[df['GrPow_MW'] != 0]
            
            # Calculate the mean of 'avg_RD' column
            GrPow_MW = df['GrPow_MW'].sum()
            
            # Append the mean, scenario, and year to the list
            data.append({'GrPow_MW': GrPow_MW, 'Scenario': scenario, 'Year': year})

# Convert the list to a DataFrame outside the loop
final_df = pd.DataFrame(data)

In [ ]:
final_df.to_csv('GrPow_MW_summary.csv', sep=';', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Filter for scenario RCP26
rcp26_data = final_df[final_df['Scenario'] == 'RCP26']

# Plotting
plt.figure(figsize=(15, 10))  # Set the figure size (optional)
plt.plot(rcp26_data['Year'], rcp26_data['GrPow_MW'], marker='o', linestyle='-')
plt.title('GrPow_MW for Scenario RCP26 Over the Years')
plt.xlabel('Year')
plt.ylabel('GrPow_MW')
plt.grid(True)
plt.xticks(rcp26_data['Year'])  # Ensure all years are marked
plt.tight_layout()  # Adjust layout to not cut off labels

# Show the plot
plt.show()


In [ ]:
import os
import pandas as pd

# Specify the directory containing the CSV files
directory = 'Data/Weibull/RCP26/'

# Initialize a list to store the data
data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('weibull_params_') and filename.endswith('.csv'):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Extract the scenario and year from the filename   
        parts = filename.split('_')
        #scenario = parts[4]  # Assumes 'RCP45' is always in the same position
        year = parts[2].split('.')[0]  # Split on '.' to remove the file extension
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep=',')
        
        # Check if 'GrPow_MW' column exists in the dataframe
        if 'shape' in df.columns:
            # Filter out rows where 'GrPow_MW' is not 0
            #df_filtered = df[df['GrPow_MW'] != 0]
            
            # Calculate the mean of 'avg_RD' column
            scale = df['shape'].mean()
            
            # Append the mean, scenario, and year to the list
            data.append({'shapemean': scale,  'Year': year})

# Convert the list to a DataFrame outside the loop
final_df = pd.DataFrame(data)

In [ ]:
final_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
# Specify the directory containing the CSV files
directory = 'Data/Results/'

# Initialize a list to store the data
data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('monthly_production_per_plz_') and filename.endswith('.csv'):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Extract the scenario and year from the filename   
        parts = filename.split('_')
        scenario = parts[4]  # Assumes 'RCP45' is always in the same position
        year = parts[5].split('.')[0]  # Split on '.' to remove the file extension
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep=';')
        
        # Check if 'GrPow_MW' column exists in the dataframe
        if 'Production' in df.columns:
            # Sum the 'GrPow_MW' column
            Production = df['Production'].sum()
            
            # Append the sum, scenario, and year to the list
            data.append({'Production': Production, 'Scenario': scenario, 'Year': year})

# Convert the list to a DataFrame outside the loop
final_df = pd.DataFrame(data)


# Filter for scenario RCP26
rcp26_data = final_df[final_df['Scenario'] == 'RCP85']

# Plotting
plt.figure(figsize=(15, 10))  # Set the figure size (optional)
plt.plot(rcp26_data['Year'], rcp26_data['Production'], marker='o', linestyle='-')
plt.title('Production Sum for Scenario RCP85 Over the Years', fontsize=32)
plt.xlabel('Year', fontsize=22)
plt.ylabel('Production Sum (TWh)', fontsize=22)

plt.grid(True)
plt.xticks(rcp26_data['Year'], fontsize= 11)  # Ensure all years are marked
plt.tight_layout()  # Adjust layout to not cut off labels
plt.savefig('Literatur/Grafiken/Wind_Production_Sum_RCP85.png')
# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
# Specify the directory containing the CSV files
directory = 'Data/Results/'

# Initialize a list to store the data
data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('PV_power_output_') and filename.endswith('.csv'):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Extract the scenario and year from the filename   
        parts = filename.split('_')
        scenario = parts[3]  # Assumes 'RCP45' is always in the same position
        year = parts[4].split('.')[0]  # Split on '.' to remove the file extension
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep=',')
        
        # Check if 'GrPow_MW' column exists in the dataframe
        if 'Production' in df.columns:
            # Sum the 'GrPow_MW' column
            Production = df['Production'].sum()
            
            # Append the sum, scenario, and year to the list
            data.append({'Production': Production, 'Scenario': scenario, 'Year': year})

# Convert the list to a DataFrame outside the loop
final_df = pd.DataFrame(data)


# Filter for scenario RCP26
rcp26_data = final_df[final_df['Scenario'] == 'RCP45']

# Plotting
plt.figure(figsize=(15, 10))  # Set the figure size (optional)
plt.plot(rcp26_data['Year'], rcp26_data['Production'], marker='o', linestyle='-')
plt.title('PV Production Sum for Scenario RCP45 Over the Years', fontsize=32)
plt.xlabel('Year', fontsize=22)
plt.ylabel('Production Sum (TWh)', fontsize=22)

plt.grid(True)
plt.xticks(rcp26_data['Year'], fontsize= 11)  # Ensure all years are marked
plt.tight_layout()  # Adjust layout to not cut off labels
plt.savefig('Literatur/Grafiken/PV_Production_Sum_RCP45.png')
# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Specify the directory containing the CSV files
directory = 'Data/wind_interim'

# Initialize a list to store the data
data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('wind_speed_probabilities_') and filename.endswith('.csv'):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Extract the scenario and year from the filename   
        parts = filename.split('_')
        scenario = parts[3]  # Assumes 'RCP45' is always in the same position
        year = parts[4].split('.')[0]  # Split on '.' to remove the file extension
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep=';')
        
        # Check if 'GrPow_MW' column exists in the dataframe
        if 'Probability' in df.columns:
            # Sum the 'GrPow_MW' column
            Probability = df['Probability'].mean()
            
            # Append the sum, scenario, and year to the list
            data.append({'Probability': Probability, 'Scenario': scenario, 'Year': year})

# Convert the list to a DataFrame outside the loop
final_df = pd.DataFrame(data)

# Display the DataFrame
final_df
# Filter for scenario RCP26
rcp26_data = final_df[final_df['Scenario'] == 'RCP26']

# Plotting
plt.figure(figsize=(15, 10))  # Set the figure size (optional)
plt.plot(rcp26_data['Year'], rcp26_data['Probability'], marker='o', linestyle='-')
plt.title('Probability sum in monthly Production for Scenario RCP26 Over the Years')
plt.xlabel('Year')
plt.ylabel('Probability')
plt.grid(True)
plt.xticks(rcp26_data['Year'])  # Ensure all years are marked
plt.tight_layout()  # Adjust layout to not cut off labels

# Show the plot
plt.show()


In [ ]:
'Data/wind_interim'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Specify the directory containing the CSV files
directory = 'Data/wind_interim'

# Initialize a list to store the data
data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('updated_plz_') and filename.endswith('.csv'):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Extract the scenario and year from the filename   
        parts = filename.split('_')
        scenario = parts[2]  # Assumes 'RCP45' is always in the same position
        year = parts[6].split('.')[0]  # Split on '.' to remove the file extension
        
        # Load the CSV file
        df = pd.read_csv(file_path, sep=';')
        
        # Check if 'GrPow_MW' column exists in the dataframe
        if 'Probability' in df.columns:
            # Sum the 'GrPow_MW' column
            Probability = df['Probability'].mean()
            
            # Append the sum, scenario, and year to the list
            data.append({'Probability': Probability, 'Scenario': scenario, 'Year': year})

# Convert the list to a DataFrame outside the loop
final_df = pd.DataFrame(data)

# Display the DataFrame
final_df
# Filter for scenario RCP26
rcp26_data = final_df[final_df['Scenario'] == 'RCP26']

# Plotting
plt.figure(figsize=(15, 10))  # Set the figure size (optional)
plt.plot(rcp26_data['Year'], rcp26_data['Probability'], marker='o', linestyle='-')
plt.title('Probability sum in monthly Production for Scenario RCP26 Over the Years')
plt.xlabel('Year')
plt.ylabel('Probability')
plt.grid(True)
plt.xticks(rcp26_data['Year'])  # Ensure all years are marked
plt.tight_layout()  # Adjust layout to not cut off labels

# Show the plot
plt.show()


In [ ]:
rcp26_data
rcp26_data['Yearly_Change'] = rcp26_data['GrPow_MW_Sum'].diff()

# Display the DataFrame to see the new column
print(rcp26_data)

In [5]:
basemap_gdf = gpd.read_file('shp/georef-germany-postleitzahl/georef-germany-postleitzahl.shp')
start_year = 2023
end_year = 2023

scenarios = ['RCP26', 'RCP45', 'RCP85']

for scenario in scenarios:
    for year in range(start_year, end_year + 1):
        base_path='Data/Results/'
        file_name = f"monthly_production_per_plz_{scenario}_{year}.csv"
        file_path = os.path.join(base_path, file_name)
        basemap_gdf['plz_code'] = basemap_gdf['plz_code'].astype(int)
    

        # Check if the file exists
        if os.path.exists(file_path):

            # Process data for each month
            for month in range(1, 13):  # Months from January to December
                monthly_data = data[data['month'] == month]
                
                # Merge with shapefile data
                merged_gdf = basemap_gdf.merge(monthly_data, how='left', left_on='plz_code', right_on='plz_code')
                
                # Plotting
                fig, ax = plt.subplots(1, 1, figsize=(15, 10))
                merged_gdf.plot(column='Production', ax=ax, legend=True,
                                legend_kwds={'label': "Production by PLZ",
                                             'orientation': "horizontal"})
                plt.title(f'Energy Production for {scenario} - {year}/{month}')
                #plt.savefig(f'Data/Results/Maps/{scenario}/{scenario}_{year}_month_{month}.png')  # Save the figure
                plt.show()


TypeError: list indices must be integers or slices, not str